# Pre-AI Email Marketing - "Spray and Pray"

We've all received these form emails where it's clear that they just spit out the same email to every person in their email list, rather than actually looking at what kind of customer you are.

Sometimes it's even worse than just slapping your first name on the pre-amble, I'm sure you've received emails like this before too:

```
Dear $fname,

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Cras semper at urna sit amet imperdiet. Integer rutrum tempor tellus et porta. Mauris elit dui, euismod a ipsum sit amet, ultricies dictum ex. Suspendisse bibendum suscipit leo sed egestas. Suspendisse potenti. Maecenas placerat lectus eget tincidunt posuere. Ut aliquet elit ante. Duis vel volutpat massa, pulvinar condimentum enim. Aliquam erat volutpat. Sed vulputate pretium mauris, eu tempus arcu lacinia sed. Aliquam lobortis massa at ligula aliquet tincidunt. Aliquam ultrices sem a ultricies pulvinar. In facilisis pretium cursus. Vestibulum et malesuada justo. Vivamus erat ex, aliquet nec porttitor eu, sodales consectetur dolor.

Sincerely,
Marketing Person
@ Company
```

Below is a simplistic example of creating that email:

In [5]:
people = ["Alice", "Bob", "Charlie", "David", "Emma"]

for person in people:
    message = f"Hey {person},\n Check out our web analytics platform, it's Awesome! It's perfect for your needs. Buy it now!\n - Marketer John"
    print(message)

Hey Alice,
 Check out our web analytics platform, it's Awesome! It's perfect for your needs. Buy it now!
 - Marketer John
Hey Bob,
 Check out our web analytics platform, it's Awesome! It's perfect for your needs. Buy it now!
 - Marketer John
Hey Charlie,
 Check out our web analytics platform, it's Awesome! It's perfect for your needs. Buy it now!
 - Marketer John
Hey David,
 Check out our web analytics platform, it's Awesome! It's perfect for your needs. Buy it now!
 - Marketer John
Hey Emma,
 Check out our web analytics platform, it's Awesome! It's perfect for your needs. Buy it now!
 - Marketer John


# ChatGPT has entered the chat

If there's something that will help you stick out as a marketer, it's when you send emails that are roughly the same in content but you use ChatGPT to generate the actual content. Below you'll see how you could prompt OpenAI for some "unique" emails to send out to folks.

## Setup OpenAI

Below we'll install OpenAI's python module, enter in our API key (which will be terminated at the end of this webinar), define the list of people (defined above), and then the system prompt.

The system prompt helps prime ChatGPT to respond with content that is helpful for us. You'll notice that we explicity request for ChatGPT to lookup the top 5 web analytics companies in order to give it a small corpus of content to work from for this demo.

In [6]:
!pip install openai --quiet

import openai

openai.api_key = "YOUR_API_KEY"

# Define the list of people

system = 'You are a helpful assistant. My name is Marketer John.\
        You help write the body of an email for a fictitious company called "Awesome Web Analytics". \
        This is a web analytics company that is similar to the top 5 web analytics companies \
        (Perform a web search to determine current top 5 web analytics companies). \
        The goal is to write a custom email to users to get them to be interested in our services. \
        The email should be less than 150 words. Address the user by name. End with my signature.'

## Loop through and request

Now we'll loop through each person and generate the emails by querying ChatGPT. Storing the responses in a python list.

**Note:** for the ten people we have listed above, the execution of this step takes about **30s**.

In [9]:

def chatgpt_generate_email(prompt, person):
    conversation = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": person},
        {"role": "assistant", "content": ""}
    ]

    # Call the OpenAI Chat API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=conversation,
        temperature=0.7,
        max_tokens=800,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    assistant_reply = response.choices[0].message['content']
    return assistant_reply

In [10]:
# Define a list to store the responses
emails = []

# Loop through each person and generate the conversation
for person in people:
    # Generate the email
    reply = chatgpt_generate_email(system, person)
    # Store the response in the dictionary
    emails.append({"person": person, "assistant_reply": reply})

## Print out the list

Now we print out the emails, with names and everything. Note that each of the responses is unique and completely random.

In [16]:
# Print the responses
for email in emails:
    person = email["person"]
    assistant_reply = email["assistant_reply"]
    print(f"Person: {person}")
    print(f"{assistant_reply}")
    print("_"*100)

Person: Alice
Hi Alice,

I hope this email finds you well. I wanted to reach out and introduce you to Awesome Web Analytics, a leading web analytics company. 

In today's digital age, understanding your website's data is crucial for success. With our advanced analytics tools, we can help you gain valuable insights into your website's performance, user behavior, and conversion rates. 

Our team of experts will provide you with comprehensive reports and recommendations to optimize your website and drive more traffic and sales. Whether you're a small business or a large enterprise, we have the right solution for you.

I would love to schedule a call or meeting to discuss how Awesome Web Analytics can help your business grow. Please let me know a convenient time for you, and I'll be happy to accommodate.

Looking forward to hearing from you soon.

Best regards,

Marketer John
Awesome Web Analytics
www.awesomewebanalytics.com
Email: john@awesomewebanalytics.com
Phone: 123-456-7890
_________

# Customizing the content to user behavior

Leveling up your game can be done pretty easily, if you have access to customer behavior data inside your app. For instance, what if a user signed up for your service but hadn't finished installing the analytics tracking code on their website?

We have a Mongo Database populated with some fake user data that looks something like this:

``` json
{
    '_id': ObjectId('64afb3fda9295d8421e7a19f'), 
    'first_name': 'James', 
    'last_name': 'Villanueva', 
    'company_name': 'Foley-Turner', 
    'stage': 'generating a tracking code', 
    'created_date': datetime.datetime(2023, 6, 26, 0, 0)
}
```

Take particular note of the key `stage`, we will be using that to help identify user behavior and have ChatGPT enhance our emails.

Below we use MongoDB as a source for user behavior, then use that to have ChatGPT write some more useful emails for the users.

In [17]:
!pip3 install pymongo --quiet

from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
mongo_db = client['webinar-mktg-email-demo']
collection = mongo_db['customers']

stages = [
    "getting started",
    "generating a tracking code",
    "adding tracking to your website",
    "real-time analytics",
    "conversion tracking",
    "funnels",
    "user segmentation",
    "custom event tracking",
    "data export",
    "dashboard customization"
]

def find_next_stage(current_stage):
    current_index = stages.index(current_stage)
    if current_index < len(stages) - 1:
        return stages[current_index + 1]
    else:
        return None

## Loop through users in collection

Limiting to 3 users for the demo, we will loop through each record in our customer collection in Mongo.

In [20]:
limit = 3
count = 0
emails = []

for record in collection.find():
    if count != limit:
        fname = record['first_name']
        stage = record['stage']

        next_stage = find_next_stage(stage)
        system = 'You are a helpful assistant, who works for me, Marketer John at Awesome Web Analytics. \
            You help write the body of an email for a fictitious company called "Awesome Web Analytics". \
            We are a web analytics company that is similar to the top 5 web analytics companies \
            (Perform a web search to determine current top 5 web analytics companies). \
            We have users that are at various stages of the pipeline of using our product and we want to \
            send them helpful emails to get them to use our product more. \
            Please write an email for {} who is on stage {} of the on-boarding process. The next stage is {}. \
            Ensure that the email describes the benefits of moving to the next stage. \
            Limit the email to 1 paragraph. End email with my signature.'.format(fname, stage, next_stage)
        
        reply = chatgpt_generate_email(system, person)
        emails.append({"fname": fname, "stage": stage, "next_stage": next_stage, "email": reply})
        
        count += 1

Now let's take a look at what those emails look like now:

In [22]:
# Print the responses
for e in emails:
    print(f"First Name: {e['fname']}\n")
    print(f"Stage: {e['stage']}\n")
    print(f"Next Stage: {e['next_stage']}\n")
    print(f"{e['email']}\n")
    print("_"*100)

First Name: James

Stage: generating a tracking code

Next Stage: adding tracking to your website

Hi James,

I hope you're doing well! I wanted to reach out and remind you about the next stage in our on-boarding process: adding tracking to your website. By adding our tracking code to your website, you'll be able to gain valuable insights into your website's performance, track user behavior, and make data-driven decisions to improve your online presence. This step is crucial in unlocking the full potential of our web analytics tools and maximizing the benefits they can bring to your business. If you have any questions or need assistance with the tracking code implementation, please don't hesitate to reach out. 

Thanks,
Marketer John
Awesome Web Analytics

____________________________________________________________________________________________________
First Name: Jessica

Stage: adding tracking to your website

Next Stage: real-time analytics

Hi Jessica,

I hope you're doing well!

# What about adding in vectors?

It'd be useful to be able to drive users to your documentation to ensure that they are able to complete what they're working on. So let's use a Vector Database (Pinecone) to store documentation, then perform a query against it to determine the best place in our documentation to send our users.

## Setup Pinecone Client

In [29]:
# FIXME: Move to a pre-requisites section to run before the webinar
from tqdm.autonotebook import tqdm
!pip install pinecone-client --quiet

pinecone_api_key = "YOUR_API_KEY"

import pinecone      

pinecone.init(      
	api_key=pinecone_api_key,      
	environment='us-west4-gcp-free'      
)

pine_index = pinecone.Index('web-app-docs')

## Create Embeddings for our Stages

We need embeddings for each of the stages in our pipeline, so we can perform searches in Pinecone for the best docs.

In [36]:
model_id = 'text-embedding-ada-002'

def getEmbeddings(text, model_id):
    try:
            text = text.replace("\n", " ")
            response = openai.Embedding.create(input=text,model=model_id)

            embedding = response['data'][0]['embedding']
            tokens = response['usage']['total_tokens']
            status = 'success'
            return embedding,tokens,status
    except Exception as e:
        print(e)
        embedding = ''
        tokens = 0
        status = 'failed'
        return embedding,tokens,status

In [42]:
stages_w_embed = []

for s in stages:
    embedding,tokens,status = getEmbeddings(s, model_id)
    stages_w_embed.append({"stage": s, "embedding": embedding})

## Search Pinecone for Closest Result

Using the embeddings for each stage, generated in the last step, we can search the Pinecone index for the closest matching document in our docs.

**Note:** This will take about **1m** to run.

In [43]:
def search_pinecone(embedding):
    response = pine_index.query(queries=[embedding], top_k=1, namespace='docs', include_metadata=True)
    metadata = response['results'][0]['matches'][0]['metadata']
    content = metadata['content']
    permalink = metadata['parent']
    return content, permalink

In [44]:
limit = 5
count = 0
emails = []

for record in collection.find():
    if count != limit:
        fname = record['first_name']
        stage = record['stage']

        next_stage = find_next_stage(stage)
        this_stage = next((item for item in stages_w_embed if item['stage'] == stage), None)
        next_stage = next((item for item in stages_w_embed if item['stage'] == next_stage), None)

        cur_content, cur_permalink = search_pinecone(this_stage['embedding'])
        next_content, next_permalink = search_pinecone(next_stage['embedding'])

        cur_link = f'https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{cur_permalink}.md'
        next_link = f'https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{next_permalink}.md'
        
        system = 'You are a helpful assistant. I am Marketer John at Awesome Web Analytics. \
            We are similar to the current top web analytics companies \
            We have users that are at various stages in using our product and we want to \
            send them helpful emails to get them to use our product more. \
            Write an email for {} who is on stage {} of the on-boarding process. The next stage is {}. \
            Ensure the email describes the benefits of moving to the next stage, \
            then always share this link: https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{}.md . \
            Limit the email to 1 paragraph. \
            End email with my signature "Best Regards, \n Marketer John.'.format(fname, stage, next_stage['stage'], next_permalink)

        reply = chatgpt_generate_email(system, person)
        emails.append({"fname": fname, "stage": stage, "next_stage": next_stage['stage'], "email": reply})
        
        count += 1

Print out the emails

In [21]:
for e in emails:
    print(f"First Name: {e['fname']}\n")
    print(f"Stage: {e['stage']}\n")
    print(f"Next Stage: {e['next_stage']}\n")
    print(f"{e['email']}\n")
    print("_"*100)

First Name: Erin

Stage: user segmentation

Next Stage: custom event tracking

Hi Erin,

I hope you're doing well! I wanted to reach out and let you know about the next stage in our onboarding process for Awesome Web Analytics. Moving on to custom event tracking can greatly enhance your experience with our product. By implementing custom event tracking, you'll gain deeper insights into user behavior and engagement, allowing you to make more data-driven decisions for your marketing strategies. To help you get started, I recommend checking out this helpful guide on custom event tracking: [link to the guide]. If you have any questions or need assistance, please don't hesitate to reach out. 

Best Regards,
Marketer John

____________________________________________________________________________________________________
First Name: Michael

Stage: adding tracking to your website

Next Stage: real-time analytics

Hi Michael,

I hope you're doing well. I wanted to reach out and congratulate 

# Lets make this easier with SingleStoreDB

### Ingest Data Live into SingleStoreDB

Before we go on to the steps of querying your data and simplifying the process down to one database, let's ingest the Mongo data into SingelStoreDB.

First we will create a link between the Mongo Atlas hosts that hold our data and our SingleStoreDB. The commands below were run inside of a notebook before the webinar.

``` sql
%%sql


CREATE LINK mongo AS MONGODB
CONFIG '{"mongodb.hosts":"ac-oazumcn-shard-00-01.jiuwtzy.mongodb.net:27017,ac-oazumcn-shard-00-00.jiuwtzy.mongodb.net:27017,ac-oazumcn-shard-00-02.jiuwtzy.mongodb.net:27017",
         "collection.include.list": "webinar-mktg-email-demo.customers",
         "mongodb.ssl.enabled":"true",
         "mongodb.authsource":"admin",
         "mongodb.members.auto.discover": "false"}'
CREDENTIALS '{"mongodb.user":"admin",
              "mongodb.password":"gPC7hcUXJtDYvnya"}';
```

Now let's create the table we need

``` sql
%%sql

CREATE TABLES AS INFER PIPELINE AS LOAD DATA LINK mongo '*' FORMAT AVRO;
```

Finally, we start the pipeline between MongoDB and SingleStoreDB.

``` sql
%%sql

START PIPELINE customers;
```

### Model Customers table and Connect to SingleStore

In [54]:
from sqlalchemy import create_engine, Column, String, select
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import func
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.orm import Session

# Define the Customer model
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'
    _id = Column(JSONB, primary_key=True)
    _more = Column(JSONB)


# Create an engine and session

conn = create_engine(connection_url)
Session = sessionmaker(bind=conn)
session = Session()

### Search Function for S2

In [57]:
def search_s2(vector):
    query = """
    select content, parent
    from (
            select content, parent,
            DOT_PRODUCT(embedding, JSON_ARRAY_PACK('{}')) as score
            from docs_splits
            order by score desc
            limit 1
    ) results
    """.format(str(vector))
    result = session.execute(query)
    return result.fetchone()

### Loop through Customers

Loop through the customers and select 3 to create emails for. 

In [59]:
limit = 5
count = 0
emails = []

# Query the Customer model using SQLAlchemy
customers_query = session.query(Customer._more).limit(limit)

for customer in customers_query:
    if count != limit:
        # Process each customer as desired
        fname = customer._more['first_name']
        stage = customer._more['stage']        
        next_stage = find_next_stage(stage)
        
        this_stage = next((item for item in stages_w_embed if item['stage'] == stage), None)
        next_stage = next((item for item in stages_w_embed if item['stage'] == next_stage), None)
        
        cur_content, cur_permalink = search_s2(this_stage['embedding'])
        next_content, next_permalink = search_s2(next_stage['embedding'])

        cur_link = f'https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{cur_permalink}.md'
        next_link = f'https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{next_permalink}.md'
        
        system = 'You are a helpful assistant. I am Marketer John at Awesome Web Analytics. We are similar to the current top web analytics companies We have users that are at various stages in using our product and we want to send them helpful emails to get them to use our product more. Write an email for {} who is on stage {} of the on-boarding process. The next stage is {}. Ensure the email describes the benefits of moving to the next stage, then always share this link: https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{}.md . Limit the email to 1 paragraph. End email with my signature "Best Regards, \n Marketer John.'.format(fname, stage, next_stage['stage'], next_permalink)
        
        reply = chatgpt_generate_email(system, person)
        emails.append({"fname": fname, "stage": stage, "next_stage": next_stage['stage'], "email": reply})

        count += 1

# Now you have the list of emails that can be used as desired
# You can commit the changes and close the session if needed
session.commit()
session.close()

### Print out emails

In [ ]:
for e in emails:
    print(f"First Name: {e['fname']}\n")
    print(f"Stage: {e['stage']}\n")
    print(f"Next Stage: {e['next_stage']}\n")
    print(f"{e['email']}\n")
    print("_"*100)

# Conclusion

By combining traditional marketing techniques with ChatGPT and powerful database functions, you can automate the writing of effective emails that are contextually important to your users, without the need to spend hours diving into the analytics yourself. As you can see in this demonstration, you can hack this together with multiple databases (Mongo + Pinecone), or you can do this with a powerful database that couples traditional Relational and NoSQL options with Vectors, enabling you to write your queries in SQL like you're used to.